<a href="https://colab.research.google.com/github/jlrocam/my-first-binder/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Preparación del entorno
#Crear las carpetas
import os
data = "/content/data"
train_images = "/content/data/train_images"
models =  "/content/models"
os.makedirs(data, exist_ok=True)
os.makedirs(train_images, exist_ok=True)
os.makedirs(models, exist_ok=True)

In [4]:
#Subir el fichero zip con las imágenes y descomprimir
!unzip /content/data/train_images/train_images.zip -d /content/data/train_images
!rm /content/data/train_images/train_images.zip
#Comprobar que las imagenes están en /data/train_images

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: train_images/ISIC_0068028.jpg  
  inflating: train_images/ISIC_0068029.jpg  
  inflating: train_images/ISIC_0068030.jpg  
  inflating: train_images/ISIC_0068031.jpg  
  inflating: train_images/ISIC_0068032.jpg  
  inflating: train_images/ISIC_0068033.jpg  
  inflating: train_images/ISIC_0068034.jpg  
  inflating: train_images/ISIC_0068035.jpg  
  inflating: train_images/ISIC_0068036.jpg  
  inflating: train_images/ISIC_0068037.jpg  
  inflating: train_images/ISIC_0068038.jpg  
  inflating: train_images/ISIC_0068039.jpg  
  inflating: train_images/ISIC_0068040.jpg  
  inflating: train_images/ISIC_0068041.jpg  
  inflating: train_images/ISIC_0068042.jpg  
  inflating: train_images/ISIC_0068043.jpg  
  inflating: train_images/ISIC_0068044.jpg  
  inflating: train_images/ISIC_0068045.jpg  
  inflating: train_images/ISIC_0068046.jpg  
  inflating: train_images/ISIC_0068047.jpg  
  inflating: train_images/ISIC_0068048.

In [5]:
#Comprobar que el archivo metadata.csv está en /content/data
#El archivo está en train_images y se debe mover a /content/data
!mv /content/data/train_images/metadata.csv /content/data


In [1]:
#Montar el Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Directorios de datos
DATASET_DIR = "/content/data/train_images"
METADATA_FILE = "/content/data/metadata.csv"
MODEL_PATH = "/content/model/skin_lesion_model.keras"

# --- Verificar la ruta de las imágenes ---
# Listar los archivos de imagen en el directorio
image_files = [f for f in os.listdir(DATASET_DIR) if os.path.isfile(os.path.join(DATASET_DIR, f))]
print(f"Encontradas {len(image_files)} imágenes en {DATASET_DIR}")
# Imprimir los primeros 10 nombres de archivo de imagen para verificación
print("Primeros 10 nombres de archivo de imagen:", image_files[:10])

# Cargar metadatos
df = pd.read_csv(METADATA_FILE)

# --- Usando ImageDataGenerator ---
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # Dividir en entrenamiento y validación

# Filter the dataframe to include only 'Benign' and 'Malignant' in 'diagnosis_1'
df = df[df['diagnosis_1'].isin(['Benign', 'Malignant'])]
df['diagnosis_1'] = df['diagnosis_1'].astype(str)
print("Unique values in 'diagnosis_1':", df['diagnosis_1'].unique())

# Add filename extension '.jpg' to the 'isic_id' column before passing it to the generator
df['isic_id'] = df['isic_id'].apply(lambda x: x + '.jpg')

train_generator = train_datagen.flow_from_dataframe(
    df,
    directory=DATASET_DIR,
    x_col="isic_id",
    y_col="diagnosis_1",
    target_size=(128, 128), # Reducir resolución de imagen
    batch_size=16, # Reducir batch size
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    df,
    directory=DATASET_DIR,
    x_col="isic_id",
    y_col="diagnosis_1",
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

# --- Crear el modelo ---
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

# --- Compilar el modelo ---
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# --- Entrenamiento ---
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# --- Guardar modelo ---
model.save(MODEL_PATH, save_format="keras")
print("Modelo guardado en", MODEL_PATH)

# --- Liberar memoria ---
del df, train_datagen, train_generator, validation_generator
import gc
gc.collect()

Encontradas 18946 imágenes en /content/data/train_images
Primeros 10 nombres de archivo de imagen: ['ISIC_0067208.jpg', 'ISIC_0065531.jpg', 'ISIC_0071192.jpg', 'ISIC_0062062.jpg', 'ISIC_0054130.jpg', 'ISIC_0061398.jpg', 'ISIC_0071366.jpg', 'ISIC_0057496.jpg', 'ISIC_0056554.jpg', 'ISIC_0063976.jpg']
Unique values in 'diagnosis_1': ['Benign' 'Malignant']


<ipython-input-6-8c8393a9804f>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diagnosis_1'] = df['diagnosis_1'].astype(str)
<ipython-input-6-8c8393a9804f>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isic_id'] = df['isic_id'].apply(lambda x: x + '.jpg')


Found 13362 validated image filenames belonging to 2 classes.
Found 3340 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
836/836 ━━━━━━━━━━━━━━━━━━━━ 463s 551ms/step - accuracy: 0.5268 - loss: 0.6992 - val_accuracy: 0.6611 - val_loss: 0.6447
Epoch 2/10
836/836 ━━━━━━━━━━━━━━━━━━━━ 460s 550ms/step - accuracy: 0.6329 - loss: 0.6510 - val_accuracy: 0.6731 - val_loss: 0.6204
Epoch 3/10
836/836 ━━━━━━━━━━━━━━━━━━━━ 447s 535ms/step - accuracy: 0.6739 - loss: 0.6173 - val_accuracy: 0.6976 - val_loss: 0.5962
Epoch 4/10
826/836 ━━━━━━━━━━━━━━━━━━━━ 4s 480ms/step - accuracy: 0.6891 - loss: 0.6009